In [20]:
import csv
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import random

In [7]:
# Loading the dataset
data = {}
with open('/kaggle/input/state-farm-distracted-driver-detection/driver_imgs_list.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        key = row[1]
        if key in data:
            data[key].append(row[2])
        else:
            data[key] = [row[2]]

        


In [8]:
# Shuffling data
for key in data:
    random.shuffle(data[key])

In [9]:
# Saving the labels in a list
labels = list(data.keys())
labels

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [10]:
import os

os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [11]:
for emotion in labels:
  os.mkdir(os.path.join('master_data/training/', emotion))
  os.mkdir(os.path.join('master_data/testing/', emotion))

In [12]:
from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():
  train_size = int(split_size*len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]
  for image in train_images:
    source = os.path.join('/kaggle/input/state-farm-distracted-driver-detection/imgs/train',emotion, image)
    dest = os.path.join('/kaggle/working/master_data/training', emotion,image)
    copyfile(source, dest)
  for image in test_images:
    source = os.path.join('/kaggle/input/state-farm-distracted-driver-detection/imgs/train', emotion, image)
    dest = os.path.join('/kaggle/working/master_data/testing', emotion,image)
    copyfile(source, dest)

In [21]:
# Creating a base pre-trained model of ResNet50
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [22]:
# Add global average pooling layer and a dense layer for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Convert features to a single vector
x = Dense(1024, activation='relu')(x)  # Fully connected layer
x = Dense(1024,activation = 'relu')(x)
predictions = Dense(10, activation='softmax')(x)  # Output layer for classification

# Create the final model
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

In [23]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [24]:
# Compiling the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
train_dir = '/kaggle/working/master_data/training'
test_dir = '/kaggle/working/master_data/testing'

# Create an instance of ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Load the training and validation data
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')


Found 17934 images belonging to 10 classes.
Found 4490 images belonging to 10 classes.


In [26]:
# Train the model
model.fit(train_generator, validation_data=valid_generator, epochs=10)

Epoch 1/10
  2/561 ━━━━━━━━━━━━━━━━━━━━ 50s 91ms/step - accuracy: 0.0469 - loss: 3.0907   

W0000 00:00:1723810625.576541     153 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


561/561 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step - accuracy: 0.4606 - loss: 1.5727

W0000 00:00:1723810902.059482     153 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


561/561 ━━━━━━━━━━━━━━━━━━━━ 339s 538ms/step - accuracy: 0.4608 - loss: 1.5720 - val_accuracy: 0.7359 - val_loss: 0.7425
Epoch 2/10
561/561 ━━━━━━━━━━━━━━━━━━━━ 262s 462ms/step - accuracy: 0.7606 - loss: 0.6972 - val_accuracy: 0.8212 - val_loss: 0.5280
Epoch 3/10
561/561 ━━━━━━━━━━━━━━━━━━━━ 263s 463ms/step - accuracy: 0.8168 - loss: 0.5475 - val_accuracy: 0.8459 - val_loss: 0.4579
Epoch 4/10
561/561 ━━━━━━━━━━━━━━━━━━━━ 261s 460ms/step - accuracy: 0.8383 - loss: 0.4676 - val_accuracy: 0.8922 - val_loss: 0.3180
Epoch 5/10
561/561 ━━━━━━━━━━━━━━━━━━━━ 261s 460ms/step - accuracy: 0.8666 - loss: 0.3993 - val_accuracy: 0.9060 - val_loss: 0.3015
Epoch 6/10
561/561 ━━━━━━━━━━━━━━━━━━━━ 261s 460ms/step - accuracy: 0.8812 - loss: 0.3597 - val_accuracy: 0.9220 - val_loss: 0.2513
Epoch 7/10
561/561 ━━━━━━━━━━━━━━━━━━━━ 260s 459ms/step - accuracy: 0.8982 - loss: 0.3137 - val_accuracy: 0.9334 - val_loss: 0.2120
Epoch 8/10
561/561 ━━━━━━━━━━━━━━━━━━━━ 261s 460ms/step - accuracy: 0.9062 - loss: 0.28